In [7]:


import matplotlib.pyplot as plt

from chord import Chord
# load 
import sys
import time

from sklearn.model_selection import cross_val_score,train_test_split
from datetime import date
import numpy as np
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import pingouin as pg
import subprocess
import os 
from scipy.stats import ranksums
from scipy.stats import pearsonr
from scipy import stats


if sys.platform == 'darwin':
    print("Current system is macOS")
    main_fold_path = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction'
elif sys.platform == 'linux':
    print("Current system is Ubuntu")
    main_fold_path = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning'
else:
    print("Current system is neither macOS nor Ubuntu")
    
    
os.chdir(main_fold_path)

from utils.fnirs_utils import get_channel_to_region_dict
    
DATA =  np.load('allData/prognosis_mix_hb/pretreatment_response/hb_data.npy')
LABEL =  np.load('allData/prognosis_mix_hb/pretreatment_response/label.npy')
data = DATA
label = LABEL

x = np.load('allData/prognosis_mix_hb/pretreatment_response/label_hamd.npy')
reduced_percentage = (x[:, 1] - x[:, 0]) / x[:, 0]

HbO = np.transpose(data[...,0::2],(0,2,1))
# HbO = individual_normalization(HbO)
print(f'HbO: {HbO.shape}')
HbR = np.transpose(data[...,1::2],(0,2,1))
HbT = HbO + HbR


label_hamd = np.load('allData/prognosis_mix_hb/pretreatment_response/label_hamd.npy')
reduced_percentage = (label_hamd[:, 1] - label_hamd[:, 0]) / label_hamd[:, 0]

Current system is macOS
HbO: (64, 1250, 52)


| Channel | Region    | Task change(R) | Mean(R) | Task activation(R) | Task change(P) | Mean(P) | Task activation(P) |
|-------------|-----------|-----------------|------------|------------------|


In [29]:
channel_to_dict =  get_channel_to_region_dict()


HB = HbO  
y_label = reduced_percentage # reduced_percentage or label
hb_dict = {
    'HbO': HbO,
    'HbR': HbR,
    'HbT': HbT
}
# feature_extraction - task activation
def extracting_task_activation_feature(HB, task_start_index, task_end_index, feature_name, i):
    if feature_name == 'task_change': x = HB[:, task_end_index:, i].mean(axis=1) - HB[:, :task_start_index, i].mean(axis=1)
    if feature_name == 'mean': x = HB[:, :, i].mean(axis=1) 
    if feature_name == 'task_activation': x = HB[:, task_start_index:task_end_index, i].mean(axis=1) - HB[:, :task_start_index, i].mean(axis=1) - HB[:, task_end_index:, i].mean(axis=1)
    return x 
        
def get_r_and_p_of_hb_feature_using_pearsonr(HB, y_label, feature_name_arr, ch):
    result = []
    for feature_name in feature_name_arr:
        x = extracting_task_activation_feature(HB, 100, 700, feature_name, ch)
        r, p = pearsonr(x, y_label)
        result.append(r)
        result.append(p)
    return result
for hb_type, HB in hb_dict.items():
    
    print(f'Correlation of {hb_type} features and HAMD reduction percentage')
    print()
    print("| Channel | Region    | Task change(R) | Mean(R) | Task activation(R) | Task change(P) | Mean(P) | Task activation(P) |")
    print("|-------------|-----------|-----------------|------------|------------------|-----------------|------------|------------------|")
    for ch in range(52):
        region_name = channel_to_dict[ch+1]
        region_name = region_name.replace('_',' ')
        r1, p1, r2, p2, r3, p3 = get_r_and_p_of_hb_feature_using_pearsonr(HB, y_label, ['task_change', 'mean', 'task_activation'], ch)
        print(f"| {ch+1} | {region_name} | {r1:.4f} | {r2:.4f} | {r3:.4f} | {p1:.4f}{'*' if p1 < 0.05 else ''} | {p2:.4f}{'*' if p2 < 0.05 else ''} | {p3:.4f}{'*' if p3 < 0.05 else ''} |")


Correlation of HbO features and HAMD reduction percentage

| Channel | Region    | Task change(R) | Mean(R) | Task activation(R) | Task change(P) | Mean(P) | Task activation(P) |
|-------------|-----------|-----------------|------------|------------------|-----------------|------------|------------------|
| 1 | right PSFC | 0.0799 | -0.0757 | 0.0382 | 0.5300 | 0.5521 | 0.7641 |
| 2 | right PSFC | -0.2901 | 0.3266 | -0.3128 | 0.0201* | 0.0084* | 0.0119* |
| 3 | right DPC | 0.2038 | -0.0529 | 0.2108 | 0.1063 | 0.6781 | 0.0945 |
| 4 | right DPC | 0.2228 | -0.1063 | 0.2319 | 0.0767 | 0.4033 | 0.0652 |
| 5 | MPC | 0.2279 | 0.0966 | 0.0051 | 0.0701 | 0.4477 | 0.9679 |
| 6 | MPC | -0.0674 | 0.0622 | -0.0672 | 0.5966 | 0.6253 | 0.5980 |
| 7 | left DPC | 0.2303 | 0.1958 | -0.1474 | 0.0672 | 0.1210 | 0.2452 |
| 8 | left DPC | -0.2416 | 0.1947 | -0.2083 | 0.0544 | 0.1232 | 0.0985 |
| 9 | left PSFC | 0.0629 | 0.0121 | 0.0223 | 0.6213 | 0.9241 | 0.8611 |
| 10 | left PSFC | 0.0090 | 0.1653 | -0.1449